In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
df = pd.read_excel('BBC_gender_complete.xlsx')

In [4]:
df.head()

,movie,director,movie_year,critic,critic_country,critic_org,Bechdel_test,director_gender
0,Mulholland Drive,David Lynch,2001.0,Simon Abrams,US,Freelance film critic,pass,M
1,In the Mood for Love,Wong Kar-wai,2000.0,Simon Abrams,US,Freelance film critic,pass,M
2,The Tree of Life,Terrence Malick,2011.0,Simon Abrams,US,Freelance film critic,nopass,M
3,Yi Yi: A One and a Two,Edward Yang,2000.0,Simon Abrams,US,Freelance film critic,pass,M
4,Goodbye to Language,Jean-Luc Godard,2014.0,Simon Abrams,US,Freelance film critic,NaN,M


In [5]:
df['movie_year'] = df['movie_year'].astype(int)
df['Bechdel_test'] = df['Bechdel_test'].replace(['nopass'],"didn't pass")
df['Bechdel_test'] = df['Bechdel_test'].replace(['pass'],"passed")
df['Bechdel_test'] = df['Bechdel_test'].fillna(value= "unknown")
df.head()

,movie,director,movie_year,critic,critic_country,critic_org,Bechdel_test,director_gender
0,Mulholland Drive,David Lynch,2001,Simon Abrams,US,Freelance film critic,passed,M
1,In the Mood for Love,Wong Kar-wai,2000,Simon Abrams,US,Freelance film critic,passed,M
2,The Tree of Life,Terrence Malick,2011,Simon Abrams,US,Freelance film critic,didn't pass,M
3,Yi Yi: A One and a Two,Edward Yang,2000,Simon Abrams,US,Freelance film critic,passed,M
4,Goodbye to Language,Jean-Luc Godard,2014,Simon Abrams,US,Freelance film critic,unknown,M


In [7]:
df.movie.nunique()

598

In [8]:
# the same director can have more than one movie in the list
df[df.director_gender=='M'].director.nunique()

353

In [9]:
df[df.director_gender=='F'].director.nunique()

48

In [10]:
df_final = df[(df.director_gender== 'F') | (df.director_gender== 'Both')]
df_final.head()

,movie,director,movie_year,critic,critic_country,critic_org,Bechdel_test,director_gender
20,Zero Dark Thirty,Kathryn Bigelow,2012,Thelma Adams,US,Freelance film critic,passed,F
23,Stories We Tell,Sarah Polley,2012,Thelma Adams,US,Freelance film critic,passed,F
28,Frozen River,Courtney Hunt,2008,Thelma Adams,US,Freelance film critic,passed,F
42,American Psycho,Mary Harron,2000,Matthew Anderson,UK,BBC Culture,passed,F
46,Lourdes,Jessica Hausner,2009,Matthew Anderson,UK,BBC Culture,passed,F


In [12]:
df_final.Bechdel_test.value_counts(normalize=True) *100

passed         65.644172
didn't pass    17.791411
unknown        16.564417
Name: Bechdel_test, dtype: float64

In [13]:
df_final.head()

,movie,director,movie_year,critic,critic_country,critic_org,Bechdel_test,director_gender
20,Zero Dark Thirty,Kathryn Bigelow,2012,Thelma Adams,US,Freelance film critic,passed,F
23,Stories We Tell,Sarah Polley,2012,Thelma Adams,US,Freelance film critic,passed,F
28,Frozen River,Courtney Hunt,2008,Thelma Adams,US,Freelance film critic,passed,F
42,American Psycho,Mary Harron,2000,Matthew Anderson,UK,BBC Culture,passed,F
46,Lourdes,Jessica Hausner,2009,Matthew Anderson,UK,BBC Culture,passed,F


### The idea is to create a map of countries where movies from female directors were chosen among the '21st Century best movies'. We simply don't know who most of those directors are because they didn't make it to the 100 movies ranking !!
### We also analyze if those movies generally do pass or not the Bechdel test

In [15]:
df1 = df_final.groupby(['critic_country', 'director_gender', 'director', 'Bechdel_test', 'movie_year'])['movie'].value_counts().reset_index(name='count')


In [16]:
df1

,critic_country,director_gender,director,Bechdel_test,movie_year,movie,count
0,Australia,F,Jennifer Kent,passed,2014,The Babadook,1
1,Australia,F,Rita Azevedo Gomes,passed,2012,A Vingança de Uma Mulher,1
2,Austria,F,Lucrecia Martel,passed,2004,The Holy Girl,1
3,Austria,F,Maren Ade,passed,2016,Toni Erdmann,1
4,Austria,F,Valeska Grisebach,unknown,2006,Longing,1
...,...,...,...,...,...,...,...
127,US,F,Sarah Polley,passed,2011,Take This Waltz,1
128,US,F,Sarah Polley,passed,2012,Stories We Tell,4
129,US,F,Sofia Coppola,passed,2003,Lost in Translation,7
130,US,F,Sofia Coppola,passed,2010,Somewhere,2


In [17]:
df1["string"] = df1["movie"] +" (" + df1["movie_year"].astype(str)+ ") "+ "by " + df1["director"] + ": " + df1["count"].map(str) + np.where(df1["count"] >1, ' votes', ' vote')
                                                                                                                                                

### How many movies did pass the Bechdel test? The color of the country (green, orange, red) gives you a hint

In [18]:
body = df1.groupby('critic_country')['string'].apply(lambda x: "<div id='movie'><h1><b>Top Movies by Female Directors</b></h1><P>{0}</P></div>".format('</p><p> '.join(x))).reset_index(name='properties.article')


In [19]:
body

,critic_country,properties.article
0,Australia,<div id='movie'><h1><b>Top Movies by Female Di...
1,Austria,<div id='movie'><h1><b>Top Movies by Female Di...
2,Brazil,<div id='movie'><h1><b>Top Movies by Female Di...
3,Canada,<div id='movie'><h1><b>Top Movies by Female Di...
4,China,<div id='movie'><h1><b>Top Movies by Female Di...
5,Colombia,<div id='movie'><h1><b>Top Movies by Female Di...
6,Cuba,<div id='movie'><h1><b>Top Movies by Female Di...
7,Egypt,<div id='movie'><h1><b>Top Movies by Female Di...
8,France,<div id='movie'><h1><b>Top Movies by Female Di...
9,Germany,<div id='movie'><h1><b>Top Movies by Female Di...


In [20]:
body.iloc[17]['properties.article']

"<div id='movie'><h1><b>Top Movies by Female Directors</b></h1><P>A Girl Walks Home Alone at Night (2014) by Ana Lily Amirpour: 1 vote</p><p> We Need to Talk About Kevin (2011) by Lynne Ramsay: 1 vote</P></div>"

In [21]:
df2 = df_final.groupby('critic_country')['Bechdel_test'].value_counts().reset_index(name='count')
df2.head()

,critic_country,Bechdel_test,count
0,Australia,passed,2
1,Austria,passed,2
2,Austria,unknown,1
3,Brazil,didn't pass,1
4,Canada,passed,6


In [22]:
df2["string"] = df2["count"].map(str) + np.where(df2["count"] >1, ' movies', ' movie') + " " + df2['Bechdel_test']                                                                                                                                          

In [23]:
header = df2.groupby('critic_country')['string'].apply(lambda x: "<div id='test_result'><h2><b>How many did pass the Bechdel test?</b></h2><P>{0}</P></div>".format('</p><p> '.join(x))).reset_index(name='properties.headline')
header

,critic_country,properties.headline
0,Australia,<div id='test_result'><h2><b>How many did pass...
1,Austria,<div id='test_result'><h2><b>How many did pass...
2,Brazil,<div id='test_result'><h2><b>How many did pass...
3,Canada,<div id='test_result'><h2><b>How many did pass...
4,China,<div id='test_result'><h2><b>How many did pass...
5,Colombia,<div id='test_result'><h2><b>How many did pass...
6,Cuba,<div id='test_result'><h2><b>How many did pass...
7,Egypt,<div id='test_result'><h2><b>How many did pass...
8,France,<div id='test_result'><h2><b>How many did pass...
9,Germany,<div id='test_result'><h2><b>How many did pass...


In [24]:
header.iloc[17]['properties.headline']

"<div id='test_result'><h2><b>How many did pass the Bechdel test?</b></h2><P>2 movies passed</P></div>"

In [25]:
output = body.merge(header, how='left', on='critic_country')

In [26]:
output

,critic_country,properties.article,properties.headline
0,Australia,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...
1,Austria,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...
2,Brazil,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...
3,Canada,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...
4,China,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...
5,Colombia,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...
6,Cuba,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...
7,Egypt,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...
8,France,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...
9,Germany,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...


In [27]:
def rand_color():
    letters = 'ABCDEF'
    import random
    from random import randint
    this_letter = random.choice(letters)
    random_num1 = randint(0,9)
    random_num2 = randint(0,9)
    hexColor = '#' + this_letter + this_letter + str(random_num1)+ str(random_num1)+ str(random_num2)+ str(random_num2)
    return hexColor

In [28]:
output['properties.color'] = output.apply(lambda x: rand_color(), axis=1)

In [29]:
output['critic_country'] = output['critic_country'].replace(['UAE'],"United Arab Emirates")
output['critic_country'] = output['critic_country'].replace(['UK'],"United Kingdom")
output['critic_country'] = output['critic_country'].replace(['US'],"United States")


In [30]:
output.head()

,critic_country,properties.article,properties.headline,properties.color
0,Australia,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#FF6699
1,Austria,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#AA6666
2,Brazil,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#AA4499
3,Canada,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#BB7799
4,China,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#FF5577


In [31]:
output.rename(columns={'critic_country': 'properties.name'}, inplace=True)

In [32]:
output.head()

,properties.name,properties.article,properties.headline,properties.color
0,Australia,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#FF6699
1,Austria,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#AA6666
2,Brazil,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#AA4499
3,Canada,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#BB7799
4,China,<div id='movie'><h1><b>Top Movies by Female Di...,<div id='test_result'><h2><b>How many did pass...,#FF5577


In [33]:
#backup your dataframes
output.to_csv('output_dataframe_final.csv', index=False)